## Binary classification neural network from scratch

In [39]:
import os
import torch
import requests
import tarfile

from PIL import Image
from torch import tensor
from torchvision import transforms
from pathlib import Path
from dataclasses import dataclass

In [100]:
dev = torch.device('cuda')

In [101]:
url = 'https://s3.amazonaws.com/fast-ai-sample/mnist_sample.tgz'

In [102]:
#sample = requests.get(url)

In [107]:
#with open('mnist_sample.tgz', 'wb') as f:
    f.write(sample.content)

In [108]:
#def ls(path: Path):
    #return [p for p in path.glob('./*')]

In [111]:
path = Path('/home/c/projects/python')

In [112]:
#with tarfile.open((path / 'mnist_sample.tgz')) as f:
   # f.extractall(path)

In [114]:
path = path / 'mnist_sample'
ls(path)

[PosixPath('/home/c/projects/python/mnist_sample/labels.csv'),
 PosixPath('/home/c/projects/python/mnist_sample/train'),
 PosixPath('/home/c/projects/python/mnist_sample/valid')]

In [115]:
transform = transforms.ToTensor()
data = {purpose: {number: torch.stack([transform(Image.open(o)).to(dev) for o in ls((path/f'{purpose}/{number}'))])/255 \
         for number in ('3', '7')} for purpose in ('train', 'valid')}

In [131]:
threes = data['train']['3']#.reshape(data['train']['3'].shape[0], 28*28)
sevens = data['train']['7']#.reshape(data['train']['7'].shape[0], 28*28)
train_x = torch.cat((threes, sevens)).view(-1, 28*28)
train_y = tensor([0] * len(data['train']['3']) + [1] * len(data['train']['7']), device=dev).unsqueeze(1)
train_x.shape, train_y.shape

(torch.Size([12396, 784]), torch.Size([12396, 1]))

In [135]:
### ADJUST METAPARAMETERS HERE
@dataclass
class MetaParameters:
    batch_size: int
    learning_rate: float
    n_epochs: int
    
mp = MetaParameters(64, 1.01, 1)
mp

MetaParameters(batch_size=64, learning_rate=1.01, n_epochs=1)

In [136]:
dset = list(zip(train_x, train_y))
dset = [dset[i : i + mp.batch_size] for i in range(0, len(dset), mp.batch_size)]

In [137]:
model = lambda x, w, b: x@w + b

In [138]:
def mnist_loss(predictions, targets) -> float:
    #sigmoid!
    #predictions = 1 / (1 + torch.exp(predictions) * -1)
    predictions = predictions.sigmoid()
    return torch.where(targets == 0, predictions, 1 - predictions).mean()

In [139]:
def init_params(shape):
    return torch.rand(shape, requires_grad=True, dtype=torch.float, device=dev)

In [140]:
def train(dset, weights, bias):
    for i in range(mp.n_epochs):
        for batch in dset:
            for x, y in batch:
                preds = model(x, weights, bias)
                loss = mnist_loss(preds, y)
                loss.backward()

                weights.data -= weights.grad * mp.learning_rate
                bias.data -= bias.grad * mp.learning_rate
                
                weights.grad.zero_()
                bias.grad.zero_()

In [141]:
weights = init_params(28*28)
bias = init_params(1)
train(dset, weights, bias)

In [147]:
model(dset[0][0][0], weights, bias)

tensor([-4.6173], device='cuda:0', grad_fn=<AddBackward0>)

In [149]:
#Image.open(data['train']['3'][0][0])
image = data['train']['7'][7][0]
transforms.ToPILImage()(image)
#image.shape

In [151]:
hi = transforms.ToTensor()(Image.open((path/'train/3/10000.png'))).reshape(28*28).to(dev)
model(hi, weights, bias)

tensor([51.4772], device='cuda:0', grad_fn=<AddBackward0>)

In [75]:
Image.open((path/'train/3/10.png'))

In [ ]:
sorted(ls(path/'train/3/'))

[PosixPath('/home/c/projects/python/mnist_sample/train/3/10.png'),
 PosixPath('/home/c/projects/python/mnist_sample/train/3/10000.png'),
 PosixPath('/home/c/projects/python/mnist_sample/train/3/10011.png'),
 PosixPath('/home/c/projects/python/mnist_sample/train/3/10031.png'),
 PosixPath('/home/c/projects/python/mnist_sample/train/3/10034.png'),
 PosixPath('/home/c/projects/python/mnist_sample/train/3/10042.png'),
 PosixPath('/home/c/projects/python/mnist_sample/train/3/10052.png'),
 PosixPath('/home/c/projects/python/mnist_sample/train/3/1007.png'),
 PosixPath('/home/c/projects/python/mnist_sample/train/3/10074.png'),
 PosixPath('/home/c/projects/python/mnist_sample/train/3/10091.png'),
 PosixPath('/home/c/projects/python/mnist_sample/train/3/10093.png'),
 PosixPath('/home/c/projects/python/mnist_sample/train/3/10097.png'),
 PosixPath('/home/c/projects/python/mnist_sample/train/3/10099.png'),
 PosixPath('/home/c/projects/python/mnist_sample/train/3/10116.png'),
 PosixPath('/home/c/proj